In [ ]:
!pip install datasets tensorflow==2.15.0 tensorrt

In [ ]:
!pip install kaggle

In [ ]:
import transformers
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, Trainer,TFAutoModelForSequenceClassification
from transformers import TrainingArguments
from datasets import ClassLabel, Value
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import numpy as np
from datasets import load_metric
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
import os
os.makedirs('kaggle', exist_ok=True)

kaggle_json = '{"username":"shervinghaffari","key":"e727e203ca6b22f6ce89f634c5ae80e7"}'

with open('kaggle.json', 'w') as f:
    f.write(kaggle_json)

!chmod 600 kaggle.json

In [ ]:
!kaggle datasets download -d quora/question-pairs-dataset
!unzip -o question-pairs-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/quora/question-pairs-dataset
License(s): other
 96% 20.0M/20.8M [00:02<00:00, 16.2MB/s]
100% 20.8M/20.8M [00:02<00:00, 9.50MB/s]
Archive:  question-pairs-dataset.zip
  inflating: questions.csv           


In [ ]:
import pandas as pd
train_df = pd.read_csv("questions.csv")
train_df.shape

(404351, 6)

In [ ]:
train_df.dropna(inplace=True)
train_df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404348 entries, 0 to 404350
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question1     404348 non-null  object
 1   question2     404348 non-null  object
 2   is_duplicate  404348 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.3+ MB


In [ ]:
df_similar = train_df[(train_df['is_duplicate'] == 1) & (train_df['question1'] != train_df['question2'])]
df_dissimilar = train_df[train_df['is_duplicate'] == 0]

In [ ]:
df_similar_sample = df_similar.sample(n=10000,random_state=42)
df_dissimilar_sample = df_dissimilar.sample(n=10000,random_state=42)

In [ ]:
# Concatenate the indices of both samples
drop_indices = df_similar_sample.index.union(df_dissimilar_sample.index)

# Drop the concatenated indices from train_df
train_df = train_df.drop(drop_indices)

In [ ]:
train_df.count()

,0
question1,384348
question2,384348
is_duplicate,384348


In [ ]:
train_df_balanced['is_duplicate'].value_counts()

,count
is_duplicate,
1,139306
0,139306


In [ ]:
from sklearn.utils import resample

similar = train_df[(train_df['is_duplicate'] == 1)]
dissimilar = train_df[train_df['is_duplicate'] == 0]
# Downsample majority class
dissimilar_downsampled = resample(dissimilar,
                                  replace=False,    # sample without replacement
                                  n_samples=len(similar), # match minority class
                                  random_state=42) # reproducible results

# Combine minority class with downsampled majority class
train_df_balanced = pd.concat([similar, dissimilar_downsampled])


In [ ]:
train_data, test_val_data = train_test_split(train_df, test_size=0.2, random_state=100, shuffle=True)
train_data.to_csv('train_clean.csv',index=False)
test_val_data.to_csv('val_clean.csv',index=False)
# test_data.to_csv('test_clean.csv',index=False)

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train_clean.csv','valid':'val_clean.csv',})

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'is_duplicate'],
        num_rows: 307478
    })
    valid: Dataset({
        features: ['question1', 'question2', 'is_duplicate'],
        num_rows: 76870
    })
})

In [ ]:
def preprocess_function(records):
    return tokenizer(
    records['question1'],
    records['question2'],
    truncation=True,
    max_length=315,
    padding=True,
    return_tensors="pt"
)


In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/307478 [00:00<?, ? examples/s]

Map:   0%|          | 0/76870 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'is_duplicate', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 307478
    })
    valid: Dataset({
        features: ['question1', 'question2', 'is_duplicate', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 76870
    })
})

In [ ]:
from datasets import concatenate_datasets,DatasetDict

In [ ]:
new_features = encoded_dataset['train'].features.copy()
new_features["is_duplicate"] = ClassLabel(num_classes=2, names=['not_duplicate', 'duplicate'], names_file=None, id=None)
encoded_dataset['train'] = encoded_dataset['train'].cast(new_features)
encoded_dataset['train'] = encoded_dataset['train'].remove_columns(['question1','question2']).rename_column('is_duplicate', 'labels')

Casting the dataset:   0%|          | 0/307478 [00:00<?, ? examples/s]

In [ ]:
new_features = encoded_dataset['valid'].features.copy()
new_features["is_duplicate"] = ClassLabel(num_classes=2, names=['not_duplicate', 'duplicate'], names_file=None, id=None)
encoded_dataset['valid'] = encoded_dataset['valid'].cast(new_features)
encoded_dataset['valid'] = encoded_dataset['valid'].remove_columns(['question1','question2']).rename_column('is_duplicate', 'labels')

Casting the dataset:   0%|          | 0/76870 [00:00<?, ? examples/s]

In [ ]:
adamw_optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
loss_fn = SparseCategoricalCrossentropy()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = "false"

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install accelerate==0.30.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [ ]:
training_args = TrainingArguments("./fine_tuned_model", eval_strategy="epoch", save_strategy='no',
                                  report_to='none', num_train_epochs=2,
                                  per_device_train_batch_size=64,
                                  per_device_eval_batch_size=64)
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("/kaggle/working/bert_fine_tuned")

In [ ]:
! pip install -q gptcache langchain tiktoken langchain_community openai==0.28 langchain_anthropic langchain_google_genai

In [ ]:
questions1 = df_similar_sample['question1'].tolist()
questions2 = df_similar_sample['question2'].tolist()

In [ ]:
from gptcache import Cache,Config
from langchain.globals import set_llm_cache
from gptcache.adapter import openai
from langchain.cache import GPTCache
from langchain_community.callbacks import get_openai_callback
from gptcache.embedding import *
from gptcache.manager import CacheBase, VectorBase, get_data_manager,manager_factory
from gptcache.similarity_evaluation import *
from langchain.embeddings.openai import OpenAIEmbeddings
import langchain,hashlib
from gptcache.adapter.api import init_similar_cache

In [ ]:
cache_store = CacheBase('sqlite')
vector_store = VectorBase("faiss", dimension=768)

data_manager = get_data_manager(cache_store,
                               vector_store,
                                max_size=10000,
                                eviction='LRU')

In [ ]:
evaluation =  SbertCrossencoderEvaluation()

In [ ]:
def post_proc(outs, inputs):
    # Let's assume you want to take the mean of the embeddings over all tokens
    return torch.mean(outs, dim=1)

In [ ]:
def to_embeddings(data, **_):
        """Generate embedding given text input

        :param data: text in string.
        :type data: str

        :return: a text embedding in shape of (dim,).
        """
        if not isinstance(data, list):
            data = [data]
        inputs = tokenizer(
            data, padding=True, truncation=True, return_tensors="pt"
        )
        outs = model(**inputs).last_hidden_state
        emb = post_proc(outs, inputs).squeeze(0).detach().numpy()
        return np.array(emb).astype("float32")

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
cache = Cache()
cache.init(embedding_func=to_embeddings,
               data_manager=data_manager,
               similarity_evaluation=evaluation,
               config=Config(
                   similarity_threshold=0.70
                    )
               )
cache.set_openai_key()

In [ ]:
def save(self,embedding,temp, **kargs):
  pass

In [ ]:
data_manager.save = save

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

total_cache_hits = 0
saved_tokens = [0,0]

def add_to_saved_tokens(new_tokens: list):
    global saved_tokens
    saved_tokens = [a + b for a, b in zip(saved_tokens, new_tokens)]


def hit_cache(res_data,**kwargs):

  global total_cache_hits

  total_cache_hits+=1


In [ ]:
data_manager.hit_cache_callback = hit_cache

In [ ]:
def llm(prompt:str):
   return openai.ChatCompletion.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "user", "content": prompt}
        ],
        cache_obj=cache,
        max_tokens=1
        )

# Populating the cache

In [ ]:
import time
delay_data = []
for question, response in zip(df_similar_sample['question1'],df_similar_sample['answer']):
  start = time.time()
  data_manager.save(question = question,answer = response,embedding_data = encoder.to_embeddings(question))
  end = time.time()
  delay_data.append(round(end-start,2))

# Total average delay

In [ ]:
import numpy as np
np.average(delay_data)

In [ ]:
saved_tokens

In [ ]:
total_cache_hits

In [ ]:
cache_store.count()

In [ ]:
model.eval()